<a href="https://colab.research.google.com/github/UOH-Group3-Project/Hackathon_group3/blob/main/group3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# This will open a file picker dialog
uploaded = files.upload()


Saving dataset_A_testing.csv to dataset_A_testing.csv
Saving dataset_A_training.csv to dataset_A_training.csv


In [ ]:
import warnings
warnings.filterwarnings("ignore") # For suppressing warnings

import pandas as pd  # For data handling
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting graphs
import seaborn as sns  # For prettier graphs
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.ensemble import RandomForestRegressor  # For building a prediction model
from sklearn.metrics import mean_squared_error, mean_absolute_error  # For evaluating the model
from datetime import datetime  # For handling dates
from sklearn.preprocessing import StandardScaler  # For scaling data
import statsmodels.api as sm  # For statistical analysis

In [ ]:
# Load the dataset
df = pd.read_csv('dataset_A_training.csv', sep=';', low_memory=False)


In [ ]:
df.head()

,"respondent_id,flu_concern,flu_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,census_msa,household_adults,household_children,employment_sector,seasonal_vaccine"
0,"1,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,..."
1,"2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,..."
2,"3,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,..."
3,"4,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,,0.0,0.0..."
4,"5,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,..."


In [ ]:
print('Total number of data points in the dataset:', len(df))

Total number of data points in the dataset: 4756


In [ ]:
missing_values_count = df.isnull().sum()
total_missing = missing_values_count.sum()

print("Missing values per column:\n", missing_values_count)
print(f"\nTotal missing values in dataset: {total_missing}")

Missing values per column:
 respondent_id,flu_concern,flu_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,census_msa,household_adults,household_children,employment_sector,seasonal_vaccine    0
dtype: int64

Total missing values in dataset: 0


In [ ]:
dx = df.copy()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import shap
import warnings
warnings.filterwarnings("ignore")

# Fix for SHAP with numpy
if not hasattr(np, 'bool'):
    np.bool = np.bool_

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset_A_testing.csv")  # adjust path if needed

print("----- Dataset Overview -----")
print(df.head(), "\n")

print("----- Dataset Info -----")
print(df.info(), "\n")

print("----- Missing Values -----")
print(df.isnull().sum(), "\n")

print("----- Basic Statistics -----")
print(df.describe().T, "\n")

In [ ]:
# STEP 2: EXPLORATORY DATA ANALYSIS (EDA)
target_col = 'target' if 'target' in df.columns else df.columns[-1]  # assume last column is target
print(f"Target column used: {target_col}")

plt.figure(figsize=(6,4))
sns.countplot(x=target_col, data=df)
plt.title("Target Variable Distribution")
plt.show()

# Correlation heatmap for numeric variables
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Feature Correlation Heatmap")
plt.show()

# Pairplot (optional, for small datasets)
# sns.pairplot(df, hue=target_col)

In [ ]:
# STEP 3: PRE-PROCESSING
X = df.drop(target_col, axis=1)
y = df[target_col]

# Encode categorical features if any
X_encoded = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

# Feature scaling (for models like Logistic Regression)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


In [ ]:
# STEP 4: CHOOSING, TRAINING & RUNNING YOUR MODEL
# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Train Logistic Regression for comparison
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

In [ ]:
# STEP 5: PERFORMANCE ANALYSIS
# Random Forest performance
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

# Logistic Regression performance
y_pred_lr = lr_model.predict(X_test_scaled)
print("\nLogistic Regression Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))
print("Classification Report:\n", classification_report(y_test, y_pred_lr))

In [ ]:
# STEP 6: EXPLAINABLE AI (XAI) WITH SHAPLEY VALUES
# Random Forest SHAP
explainer_rf = shap.TreeExplainer(
    rf_model, X_train, feature_perturbation="interventional", check_additivity=False
)
shap_values_rf = explainer_rf.shap_values(X_test)

rf_importance = np.abs(shap_values_rf[1]).mean(axis=0)
rf_feature_importance = pd.DataFrame({
    "Feature": X_test.columns,
    "Mean |SHAP value|": rf_importance
}).sort_values(by="Mean |SHAP value|", ascending=False)

print("\nTop Features (Random Forest):")
print(rf_feature_importance.head(10))

# Summary plot for global feature importance
shap.summary_plot(shap_values_rf[1], X_test, plot_type="bar")

# Instance-level SHAP explanation
instance_index = 22
shap.initjs()
shap.plots.waterfall(
    shap.Explanation(
        values=shap_values_rf[1][instance_index],
        base_values=explainer_rf.expected_value[1],
        data=X_test.iloc[instance_index],
        feature_names=X_test.columns
    )
)

# Linear Regression SHAP
explainer_lr = shap.LinearExplainer(lr_model, X_train_scaled)
shap_values_lr = explainer_lr.shap_values(X_test_scaled)

lr_importance = np.abs(shap_values_lr).mean(axis=0)
lr_feature_importance = pd.DataFrame({
    "Feature": X_test.columns,
    "Mean |SHAP value|": lr_importance
}).sort_values(by="Mean |SHAP value|", ascending=False)

print("\nTop Features (Logistic Regression):")
print(lr_feature_importance.head(10))

# Optional summary plot for linear model
shap.summary_plot(shap_values_lr, X_test_scaled, plot_type="bar")

print("\n✅ All 6 Steps Completed Successfully!")


In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn plotly

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Load the dataset
from io import StringIO

# Since we have the data in CSV format, let's create a DataFrame
data = """respondent_id,flu_concern,flu_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,census_msa,household_adults,household_children,employment_sector,seasonal_vaccine
1,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,2.0,18 - 34 Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,Non-MSA,0.0,0.0,entertainment,0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0,4.0,18 - 34 Years,Some College,White,Male,,Not Married,Rent,Employed,"MSA, Principle City",1.0,0.0,real_estate,0
3,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,1.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,"MSA, Not Principle  City",1.0,0.0,,0
4,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,,0.0,0.0,0.0,1.0,3.0,3.0,1.0,55 - 64 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,"MSA, Not Principle  City",1.0,2.0,utilities,1
5,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,,3.0,1.0,1.0,55 - 64 Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,"MSA, Not Principle  City",2.0,0.0,,0"""

# For the full dataset, you would load from file. Since we have limited data shown,
# I'll create a function to load the actual file
def load_dataset():
    # If you have the file uploaded to Colab, use this:
    # return pd.read_csv('dataset_A_training.csv')

    # For now, I'll create a sample analysis with the data we have
    # In practice, you would upload your CSV file to Colab and use the line above
    print("Please upload your 'dataset_A_training.csv' file to Colab")
    print("Then uncomment the line: df = pd.read_csv('dataset_A_training.csv')")

    # Create a small sample dataset for demonstration
    sample_data = {
        'flu_concern': [2.0, 1.0, 2.0, 1.0, 2.0],
        'flu_knowledge': [0.0, 1.0, 1.0, 2.0, 2.0],
        'behavioral_wash_hands': [1.0, 0.0, 1.0, 1.0, 1.0],
        'age_group': ['18 - 34 Years', '18 - 34 Years', '65+ Years', '55 - 64 Years', '55 - 64 Years'],
        'education': ['12 Years', 'Some College', 'College Graduate', 'College Graduate', '12 Years'],
        'race': ['White', 'White', 'White', 'White', 'White'],
        'sex': ['Male', 'Male', 'Male', 'Male', 'Female'],
        'seasonal_vaccine': [0, 0, 0, 1, 0]
    }
    return pd.DataFrame(sample_data)

# Load data
df = load_dataset()

print("Dataset Overview:")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\nColumn names:")
print(df.columns.tolist())

print(f"\nData types:")
print(df.dtypes)

print(f"\nMissing values:")
print(df.isnull().sum())

# Basic Data Analysis
print("\n" + "="*50)
print("BASIC DATA ANALYSIS")
print("="*50)

# Target variable distribution
if 'seasonal_vaccine' in df.columns:
    plt.figure(figsize=(10, 6))

    plt.subplot(1, 2, 1)
    df['seasonal_vaccine'].value_counts().plot(kind='bar')
    plt.title('Seasonal Vaccine Distribution')
    plt.xlabel('Vaccine Status (0=No, 1=Yes)')
    plt.ylabel('Count')

    plt.subplot(1, 2, 2)
    df['seasonal_vaccine'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('Seasonal Vaccine Proportion')

    plt.tight_layout()
    plt.show()

    print(f"Target variable distribution:")
    print(df['seasonal_vaccine'].value_counts())

# Demographic Analysis
print("\n" + "="*50)